In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops

ops.reset_default_graph()

# ---------------------------------------------------|
# -------------------1D-data-------------------------|
# ---------------------------------------------------|

# Create graph session 
sess = tf.Session()
# Generate 1D data
data_size = 25
data_1d = np.random.normal(size=data_size)
print(data_1d)

[-0.95735045 -0.10964125  1.35833856  2.30459991  1.80469558 -0.06477176
 -0.28948875  0.61346719  1.2502532  -0.39251752 -0.01089193  0.01062762
 -0.48037641 -0.24640901  0.0904717  -0.35102312 -0.2872073   1.00434224
  0.46385984 -1.45436252  0.72456804  0.57099146  0.45446039 -0.3033449
  1.20884539]


In [21]:
# Placeholder
x_input_1d = tf.placeholder(shape=[data_size],dtype=tf.float32)
# Create filter for convolution.
conv_filter_1 = tf.Variable(tf.random_normal(shape=[1,5,1,1]))

# --------Convolution--------
def conv_layers_1d(input_1d,my_filter):
    input_2d = tf.expand_dims(input_1d,0)
    input_3d = tf.expand_dims(input_2d,0)
    input_4d = tf.expand_dims(input_3d,3)
    #input_4d like (1,1,input_1d,1)
    convolution_output = tf.nn.conv2d(input_4d,filter=my_filter,strides=[1,1,1,1],padding='VALID')
    #print(convolution_output) #Tensor("Conv2D_4:0", shape=(1, 1, 21, 1), dtype=float32)
    conv_output_1d = tf.squeeze(convolution_output)
    return conv_output_1d
# Create convolution layer
conv_output_1 = conv_layers_1d(x_input_1d,conv_filter_1)
# --------Activation Function--------
def activation(input_1d):
    return tf.nn.relu(input_1d)

# Create activation layer
activation_output_1 = activation(conv_output_1)
# --------Max Pool--------
def max_pool(input_1d,width):
    # Just like 'conv2d()' above, max_pool() works with 4D arrays.
    # [batch_size=1, width=1, height=num_input, channels=1]
    input_2d = tf.expand_dims(input_1d,0)
    input_3d = tf.expand_dims(input_2d,0)
    input_4d = tf.expand_dims(input_3d,3)
    pool_output = tf.nn.max_pool(input_4d,ksize=[1,1,width,1],strides=[1,1,1,1],padding='VALID')
    # Get rid of extra dimensions
    pool_output_1d = tf.squeeze(pool_output)
    return pool_output_1d

maxpool_output_1 = max_pool(activation_output_1,width=5)
# help(tf.nn.max_pool)
# --------Fully Connected--------
def fully_connected(input_layer,num_outputs):
    # First we find the needed shape of the multiplication weight matrix:
    # The dimension will be (length of input) by (num_outputs)
    # tf.pack modified to tf.stack
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer),[num_outputs]]))
    # Initialize such weight
    weight = tf.random_normal(weight_shape,stddev=0.1)
    # Initialize the bias
    bias = tf.random_normal(shape=[num_outputs])
    # Make the 1D input array into a 2D array for matrix multiplication
    input_layer_2d = tf.expand_dims(input_layer,0)
    # Perform the matrix multiplication and add the bias
    full_output = tf.add(tf.matmul(input_layer_2d,weight),bias)
    # Get rid of extra dimensions
    full_output_1d = tf.squeeze(full_output)
    return full_output_1d

full_output_1 = fully_connected(maxpool_output_1,num_outputs=5)
# Run graph
# Initialize Variables
init = tf.global_variables_initializer()
sess.run(init)
feed_dict = {x_input_1d: data_1d}


Tensor("Conv2D_4:0", shape=(1, 1, 21, 1), dtype=float32)


In [29]:
# Convolution Output
print(x_input_1d)
print(conv_filter_1,sess.run(conv_filter_1))
print(conv_output_1)
print("Convolution Output--->",sess.run(conv_output_1,feed_dict=feed_dict))

# test conv2d function
input = tf.Variable(tf.random_normal([10,5,5,5])) # shape = [batch, in_height, in_width, in_channels]
filter = tf.Variable(tf.random_normal([3,3,5,7])) # shape = [filter_height, filter_width, in_channels, out_channels]
op = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='VALID') #output shape = [batch, height, width, out_channels]
print(op)

Tensor("Placeholder_7:0", shape=(25,), dtype=float32)
<tf.Variable 'Variable_6:0' shape=(1, 5, 1, 1) dtype=float32_ref> [[[[-0.95762062]]

  [[-1.60981584]]

  [[ 0.28044611]]

  [[ 0.53298986]]

  [[ 0.68885231]]]]
Tensor("Squeeze_10:0", shape=(21,), dtype=float32)
Convolution Output---> [ 3.94571877 -0.51809794 -4.7385726  -4.86203146 -0.51691747  1.09608054
 -0.57643098 -2.70871067 -0.893686   -0.02937788 -0.21040976  0.50045335
  0.49712873  0.53064537  1.25273395  0.3255533  -1.48772287 -1.33686185
  2.71764517  0.41970158 -0.81456453]
Tensor("Conv2D_12:0", shape=(10, 3, 3, 7), dtype=float32)


In [31]:
# Activation Output
print(sess.run(activation_output_1, feed_dict=feed_dict))

# Max Pool Output
print(sess.run(maxpool_output_1, feed_dict=feed_dict))

# Fully Connected Output
print(sess.run(full_output_1, feed_dict=feed_dict))

[ 3.94571877  0.          0.          0.          0.          1.09608054
  0.          0.          0.          0.          0.          0.50045335
  0.49712873  0.53064537  1.25273395  0.3255533   0.          0.
  2.71764517  0.41970158  0.        ]
[ 3.94571877  1.09608054  1.09608054  1.09608054  1.09608054  1.09608054
  0.          0.50045335  0.50045335  0.53064537  1.25273395  1.25273395
  1.25273395  1.25273395  2.71764517  2.71764517  2.71764517]
[ 0.08316433  0.90245897 -0.26357979 -0.8294543  -0.62079698]


In [32]:
# ---------------------------------------------------|
# -------------------2D-data-------------------------|
# ---------------------------------------------------|

# Reset Graph
ops.reset_default_graph()
sess = tf.Session()

# Generate 2D data
data_size = [10, 10]
data_2d = np.random.normal(size=data_size)

# --------Placeholder--------
x_input_2d = tf.placeholder(dtype=tf.float32, shape=data_size)


# Convolution
def conv_layer_2d(input_2d, my_filter):
    # Tensorflow's 'conv2d()' function only works with 4D arrays:
    # [batch#, width, height, channels], we have 1 batch, and
    # 1 channel, but we do have width AND height this time.
    # So next we create the 4D array by inserting dimension 1's.
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Note the stride difference below!
    convolution_output = tf.nn.conv2d(input_4d, filter=my_filter, strides=[1, 2, 2, 1], padding="VALID")
    # Get rid of unnecessary dimensions
    conv_output_2d = tf.squeeze(convolution_output)
    return conv_output_2d


# Create Convolutional Filter
my_filter = tf.Variable(tf.random_normal(shape=[2, 2, 1, 1]))
# Create Convolutional Layer
my_convolution_output = conv_layer_2d(x_input_2d, my_filter)


# --------Activation--------
def activation(input_1d):
    return tf.nn.relu(input_1d)


# Create Activation Layer
my_activation_output = activation(my_convolution_output)


# --------Max Pool--------
def max_pool(input_2d, width, height):
    # Just like 'conv2d()' above, max_pool() works with 4D arrays.
    # [batch_size=1, width=given, height=given, channels=1]
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    # Perform the max pooling with strides = [1,1,1,1]
    # If we wanted to increase the stride on our data dimension, say by
    # a factor of '2', we put strides = [1, 2, 2, 1]
    pool_output = tf.nn.max_pool(input_4d, ksize=[1, height, width, 1],
                                 strides=[1, 1, 1, 1],
                                 padding='VALID')
    # Get rid of unnecessary dimensions
    pool_output_2d = tf.squeeze(pool_output)
    return pool_output_2d


# Create Max-Pool Layer
my_maxpool_output = max_pool(my_activation_output, width=2, height=2)


# --------Fully Connected--------
def fully_connected(input_layer, num_outputs):
    # In order to connect our whole W byH 2d array, we first flatten it out to
    # a W times H 1D array.
    flat_input = tf.reshape(input_layer, [-1])
    # We then find out how long it is, and create an array for the shape of
    # the multiplication weight = (WxH) by (num_outputs)
    # tf.pack modified to tf.stack
    weight_shape = tf.squeeze(tf.stack([tf.shape(flat_input), [num_outputs]]))
    # Initialize the weight
    weight = tf.random_normal(weight_shape, stddev=0.1)
    # Initialize the bias
    bias = tf.random_normal(shape=[num_outputs])
    # Now make the flat 1D array into a 2D array for multiplication
    input_2d = tf.expand_dims(flat_input, 0)
    # Multiply and add the bias
    full_output = tf.add(tf.matmul(input_2d, weight), bias)
    # Get rid of extra dimension
    full_output_2d = tf.squeeze(full_output)
    return full_output_2d



In [33]:
# Create Fully Connected Layer
my_full_output = fully_connected(my_maxpool_output, 5)

# Run graph
# Initialize Variables
init = tf.initialize_all_variables()
sess.run(init)

feed_dict = {x_input_2d: data_2d}

# Convolution Output
print(sess.run(my_convolution_output, feed_dict=feed_dict))

# Activation Output
print(sess.run(my_activation_output, feed_dict=feed_dict))

# Max Pool Output
print(sess.run(my_maxpool_output, feed_dict=feed_dict))

# Fully Connected Output
print(sess.run(my_full_output, feed_dict=feed_dict))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[-0.17095304  0.22788841  0.5058319   0.50707757  0.42798865]
 [ 0.03318463 -0.13764277 -0.20564047  0.01782988  0.69035554]
 [-0.15543161 -0.02368833 -0.27016124  0.86132699  0.90642607]
 [ 0.10998848  0.58655697 -0.06125756 -0.64230275 -0.38151535]
 [-1.21721959  0.90973336  1.09701717 -0.04690319 -0.36419433]]
[[ 0.          0.22788841  0.5058319   0.50707757  0.42798865]
 [ 0.03318463  0.          0.          0.01782988  0.69035554]
 [ 0.          0.          0.          0.86132699  0.90642607]
 [ 0.10998848  0.58655697  0.          0.          0.        ]
 [ 0.          0.90973336  1.09701717  0.          0.        ]]
[[ 0.22788841  0.5058319   0.50707757  0.69035554]
 [ 0.03318463  0.          0.86132699  0.90642607]
 [ 0.58655697  0.58655697  0.86132699  0.90642607]
 [ 0.90973336  1.09701717  1.09701717  0.        ]]
[ -2.73105294e-01   1.90842509e-01  -3.70442867e-04  -4.80848402e-01
   1.76736876e-01]
